In [1]:
import gensim
from gensim import models

In [11]:
# 预训练的词向量语料库文件路径
tecent_embedding_path = (r'D:\embedding_zh\tencent-ailab-embedding-zh-d200-v0.2.0-s\tencent-ailab-embedding-zh-d200-v0.2.0-s.txt')

# 加载词向量文件
tencent_embedding = models.KeyedVectors.load_word2vec_format(tecent_embedding_path, binary=False)

In [12]:
# 导入读取数据所需要的库
import numpy as np
import pandas as pd
from pandas import read_excel

df = read_excel(r'data\SEV_wordlist.xlsx')
# 查看数据的形状
df.shape

(256, 23)

In [13]:
# 查看数据前五行
df.head()

,Word,Dimension,App_Dim_mean,App_Dim_sd,SES_Dim_mean,SES_Dim_sd,Soc_Dim_mean,Soc_Dim_sd,Com_Dim_mean,Com_Dim_sd,...,Val_sd,Valence,N_raters,N_App,N_SES,N_Soc,N_Com,N_Mor,N_Non,N_Amb
0,美,Appearance,8.822,0.736,2.425234,2.009723,2.242991,1.992197,2.780374,2.505091,...,1.123,Positive,214,182,1,0,1,3,0,27
1,帅,Appearance,8.795,0.940,2.120930,1.883097,2.669767,2.351728,2.762791,2.421576,...,1.281,Positive,215,193,0,1,0,0,2,19
2,漂亮,Appearance,8.740,1.109,2.781395,2.356861,3.246512,2.584945,3.390698,2.711331,...,1.260,Positive,215,189,1,1,1,0,3,20
3,好看,Appearance,8.701,0.942,1.957944,1.715192,2.028037,1.738587,2.364486,2.238764,...,1.256,Positive,214,201,2,0,0,0,1,10
4,美丽,Appearance,8.668,1.255,2.126168,1.812312,2.130841,1.838984,2.331776,2.073128,...,1.201,Positive,214,191,0,1,1,2,4,15


In [14]:
# 求积极词汇的词表
df_positive = df.loc[df['Valence']=='Positive', 'Word']
df_positive.head()

0     美
1     帅
2    漂亮
3    好看
4    美丽
Name: Word, dtype: object

In [15]:
# 格式转化为列表
positive_word_list = df_positive.tolist()
positive_word_list


['美',
 '帅',
 '漂亮',
 '好看',
 '美丽',
 '天香国色',
 '花容月貌',
 '天姿国色',
 '帅气',
 '眉清目秀',
 '美女',
 '俊美',
 '苗条',
 '耐看',
 '俊秀',
 '俊俏',
 '靓女',
 '靓丽',
 '英俊',
 '白净',
 '可爱',
 '沉鱼落雁',
 '迷人',
 '亭亭玉立',
 '清秀',
 '白',
 '俏丽',
 '国色天香',
 '美貌',
 '高挑儿',
 '妩媚',
 '闭月羞花',
 '清纯',
 '性感',
 '高大',
 '细嫩',
 '童颜鹤发',
 '惊艳',
 '面善',
 '丰腴',
 '甜美',
 '一表人才',
 '强壮',
 '婀娜',
 '女神',
 '文质彬彬',
 '鹤发童颜',
 '健壮',
 '干净',
 '秀色可餐',
 '圆润',
 '挺拔',
 '柔美',
 '身强力壮',
 '西施',
 '姣好',
 '娇艳',
 '年轻',
 '光彩照人',
 '壮实',
 '匀称',
 '光鲜',
 '笔挺',
 '小巧玲珑',
 '端庄',
 '精致',
 '壮健',
 '阳光',
 '稚气',
 '骨感',
 '明眸皓齿',
 '魅力',
 '衣冠楚楚',
 '顺眼',
 '斯文',
 '风华正茂',
 '娇柔',
 '精壮',
 '淑女',
 '粗壮',
 '威猛',
 '雪白',
 '秀外慧中',
 '强健',
 '书生气',
 '温文尔雅',
 '珠圆玉润',
 '时髦',
 '整洁',
 '明眸',
 '慈祥',
 '彪悍',
 '倜傥',
 '优雅',
 '俏皮',
 '剽悍',
 '文静',
 '素雅',
 '雍容',
 '儒雅',
 '憨厚',
 '憨实',
 '玉洁冰清',
 '饱经风霜',
 '结实',
 '阳刚',
 '硬朗',
 '气质',
 '上相',
 '飒爽',
 '优美',
 '健美',
 '文雅',
 '清新',
 '冰清玉洁',
 '和蔼',
 '时尚',
 '英武',
 '文气',
 '乖巧',
 '朝气',
 '精神',
 '正点',
 '神气',
 '典雅',
 '意气风发',
 '恬静',
 '朴素',
 '威武',
 '独特',
 '短小精悍',
 '灵透',
 '柔和',
 '吉人天相',
 '凝脂',


In [17]:
# 初始化有效种子词列表和相似词列表
positive_word_list_effective = []
similar_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in positive_word_list:
    if word in tencent_embedding:
        positive_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        similar_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_word, similarity in similar_words:
            similar_words_list.append(similar_word)
    else:
        print(f"Skipping {word}...")

Skipping 高挑儿...


In [18]:
# 将生成的积极相似词先转换为 DataFrame，查看是否生成成功
df_similar_words = pd.DataFrame(similar_words_list, columns=['Word'])
print(df_similar_words)

      Word
0       的美
1       美的
2       和美
3       叫美
4        丽
...    ...
1565    虬髯
1566  黑脸汉子
1567  中年汉子
1568    莽汉
1569    魁伟

[1570 rows x 1 columns]


In [19]:
# 将相似词列表转换为集合再转换回列表，以去掉重复出现的词
unique_similar_words_list = list(set(similar_words_list))

In [21]:
# 将去重后的结果转换为 DataFrame
df_unique_similar_words = pd.DataFrame(unique_similar_words_list, columns=['Word'])
print(df_unique_similar_words)

      Word
0       厚实
1     仙风道骨
2     阳刚之美
3       威猛
4       精美
...    ...
1238    清丽
1239    有型
1240  蕙质兰心
1241  青春正好
1242  长得英俊

[1243 rows x 1 columns]


In [23]:
# 保存去重后的相似词表到excel文件
df_unique_similar_words.to_excel(r'result\positive_similar_words_unique.xlsx', index=False)

下面做消极词汇的表：

In [25]:
# 求消极词汇的词表
df_negative = df.loc[df['Valence']=='Negative', 'Word']
df_negative.head()

158       丑
159       胖
160      黢黑
161      丑陋
162    胡子拉碴
Name: Word, dtype: object

In [26]:
# 格式转化为列表
negative_word_list = df_negative.tolist()
negative_word_list

['丑',
 '胖',
 '黢黑',
 '丑陋',
 '胡子拉碴',
 '丑八怪',
 '矮小',
 '难看',
 '肥胖',
 '膀大腰圆',
 '面黄肌瘦',
 '矮',
 '黑黢黢',
 '瘦小',
 '黑黝黝',
 '苍老',
 '秃顶',
 '其貌不扬',
 '消瘦',
 '黑',
 '妖艳',
 '贼眉鼠眼',
 '蓬头垢面',
 '尖嘴猴腮',
 '瘦弱',
 '驼背',
 '妖媚',
 '虎背熊腰',
 '娇媚',
 '浓艳',
 '灰头土脸',
 '邋遢',
 '皱纹',
 '纤弱',
 '憔悴',
 '娇娆',
 '土气',
 '老气',
 '凶巴巴',
 '娇滴滴',
 '獐头鼠目',
 '油腻',
 '不修边幅',
 '青面獠牙',
 '狰狞',
 '凶恶',
 '丑恶',
 '贼头贼脑',
 '粗犷',
 '弱不禁风',
 '凶悍',
 '艳俗',
 '囚首垢面',
 '风骚',
 '文弱',
 '呆头呆脑',
 '孱弱',
 '慓悍',
 '呆傻',
 '凶狠',
 '病弱',
 '凶横',
 '阴柔',
 '脏',
 '魅惑',
 '冷若冰霜',
 '老气横秋',
 '尖嘴',
 '傻乎乎',
 '可怕',
 '弱小',
 '病态',
 '愣头愣脑',
 '凶猛',
 '羞羞答答',
 '冷酷',
 '呆滞',
 '虚弱',
 '呆板',
 '恶煞',
 '奇怪',
 '褴褛',
 '冷傲',
 '清冷',
 '羸弱',
 '无精打采',
 '病病歪歪',
 '媚俗',
 '古怪',
 '傻呵呵',
 '阴郁',
 '粗糙',
 '阴沉',
 '凶狂',
 '萎靡',
 '野性',
 '死气沉沉',
 '滑稽']

In [27]:
# 初始化有效种子词列表和相似词列表
negative_word_list_effective = []
similar_negative_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in negative_word_list:
    if word in tencent_embedding:
        negative_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        similar_negative_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_negative_word, similarity in similar_negative_words:
            similar_negative_words_list.append(similar_negative_word)
    else:
        print(f"Skipping {word}...")

Skipping 囚首垢面...
Skipping 慓悍...
Skipping 病病歪歪...


In [28]:
# 将生成的积极相似词先转换为 DataFrame，查看是否生成成功
df_similar_negative_words = pd.DataFrame(similar_negative_words_list, columns=['Word'])
print(df_similar_negative_words)

     Word
0      丑的
1     长得丑
2     长的丑
3    长得难看
4      不丑
..    ...
945  惹人发笑
946    可笑
947  十分好笑
948  引人发笑
949    发噱

[950 rows x 1 columns]


In [29]:
# 将相似词列表转换为集合再转换回列表，以去掉重复出现的词
unique_similar_negative_words_list = list(set(similar_negative_words_list))

In [30]:
# 将去重后的结果转换为 DataFrame
df_unique_similar_negative_words = pd.DataFrame(unique_similar_negative_words_list, columns=['Word'])
print(df_unique_similar_negative_words)

     Word
0     黑黢黢
1    凶悍无比
2    流里流气
3      悍勇
4    满脸胡茬
..    ...
734    单薄
735   板着脸
736  脸色憔悴
737  牛高马大
738  傻不拉几

[739 rows x 1 columns]


In [31]:
# 保存去重后的相似词表到excel文件
df_unique_similar_negative_words.to_excel(r'result\negative_similar_words_unique.xlsx', index=False)